In [ ]:
import os
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical
     
# Load and preprocess MIDI files
def get_notes(data_folder):
    notes = []
    for file in os.listdir(data_folder):
        if file.endswith(".mid"):
            midi = converter.parse(os.path.join(data_folder, file))
            parts = instrument.partitionByInstrument(midi)
            if parts:  # File has instrument parts
                notes_to_parse = parts.parts[0].recurse()
            else:  # File has no instrument parts
                notes_to_parse = midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes    

# Prepare input sequences for the model
def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in seq_in])
        network_output.append(note_to_int[seq_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output, num_classes=n_vocab)
    return network_input, network_output

# Prepare input sequences for the model
def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in seq_in])
        network_output.append(note_to_int[seq_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output, num_classes=n_vocab)
    return network_input, network_output
     
# Build the LSTM model
def build_model(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Generate new music
def generate_notes(model, network_input, pitchnames, n_vocab):
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []
    for note_index in range(500):  # Generate 500 notes
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:]
    return prediction_output

# Convert generated notes to MIDI
def create_midi(prediction_output, output_file="output.mid"):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if '.' in pattern or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = [note.Note(int(n)) for n in notes_in_chord]
            for n in notes:
                n.storedInstrument = instrument.Piano()
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

# Main execution
DATA_FOLDER = "lofi-music"
notes = get_notes(DATA_FOLDER)
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)
pitchnames = sorted(set(notes))

model = build_model(network_input, n_vocab)
model.fit(network_input, network_output, epochs=100, batch_size=64)

prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
create_midi(prediction_output)
print("Music generated and saved as output.mid!")

C:\Users\Prane\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 195s 2s/step - loss: 4.5771
Epoch 2/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - loss: 4.1411
Epoch 3/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 212s 3s/step - loss: 4.0549
Epoch 4/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 240s 3s/step - loss: 4.0659
Epoch 5/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 264s 3s/step - loss: 4.0045
Epoch 6/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 303s 4s/step - loss: 3.9808
Epoch 7/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 287s 4s/step - loss: 3.9458
Epoch 8/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 238s 3s/step - loss: 3.9488
Epoch 9/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 225s 3s/step - loss: 4.0887
Epoch 10/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 215s 3s/step - loss: 3.8683
Epoch 11/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 194s 3s/step - loss: 3.8075
Epoch 12/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - loss: 3.7584
Epoch 13/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - loss: 3.6888
Epoch 14/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - loss: 3.8690
Epoch 15/100
77/77 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - lo

In [ ]:
import os
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation

# Function to get all MIDI files from a folder
def get_midi_files(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"The specified folder does not exist: {path}")
    
    midi_files = [f for f in os.listdir(path) if f.endswith(".mid")]
    if not midi_files:
        raise ValueError(f"No MIDI files found in folder: {path}. Please add valid `.mid` files.")
    
    print(f"Found {len(midi_files)} MIDI files in {path}.")
    return [os.path.join(path, midi) for midi in midi_files]

# Function to extract notes and chords from MIDI files
def extract_notes(midi_paths):
    notes = []
    for midi_path in midi_paths:
        try:
            midi = converter.parse(midi_path)
            parts = instrument.partitionByInstrument(midi)
            notes_to_parse = parts.parts[0].recurse() if parts else midi.flat.notes
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"Error processing {midi_path}: {e}")
    if not notes:
        raise ValueError("No notes extracted from the MIDI files.")
    return notes

# Prepare sequences of notes for training
def prepare_sequences(notes, sequence_length):
    note_names = sorted(set(notes))
    note_to_int = {note: number for number, note in enumerate(note_names)}
    int_to_note = {number: note for note, number in note_to_int.items()}
    
    network_input = []
    network_output = []
    for i in range(len(notes) - sequence_length):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[note] for note in sequence_in])
        network_output.append(note_to_int[sequence_out])

    if not network_input or not network_output:
        raise ValueError("Sequences could not be generated. Check the dataset size or sequence length.")
    
    X = np.reshape(network_input, (len(network_input), sequence_length, 1))
    X = X / float(len(note_names))  # Normalize input data
    y = np.zeros((len(network_output), len(note_names)))
    for i, output in enumerate(network_output):
        y[i][output] = 1

    return X, y, note_to_int, int_to_note

# Build the LSTM model
def build_model(input_shape, output_size):
    model = Sequential()
    model.add(LSTM(512, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(output_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

# Generate music using the trained model
def generate_music(model, sequence_length, note_to_int, int_to_note, seed=None, num_notes=500):
    prediction_input = seed or network_input[np.random.randint(0, len(network_input)-1)]
    prediction_output = []

    for _ in range(num_notes):
        input_reshaped = np.reshape(prediction_input, (1, len(prediction_input), 1))
        input_reshaped = input_reshaped / float(len(note_to_int))
        predicted_probs = model.predict(input_reshaped, verbose=0)
        index = np.argmax(predicted_probs)
        result = int_to_note[index]
        prediction_output.append(result)
        prediction_input.append(index)
        prediction_input = prediction_input[1:]

    return prediction_output

# Create a MIDI file from generated notes
def create_midi(notes, file_name="generated_music.mid"):
    midi_stream = stream.Stream()
    for pattern in notes:
        if '.' in pattern or pattern.isdigit():
            chord_notes = [note.Note(int(n)) for n in pattern.split('.')]
            midi_stream.append(chord.Chord(chord_notes))
        else:
            midi_stream.append(note.Note(pattern))
    midi_stream.write('midi', fp=file_name)

# Main script
try:
    midi_folder = r"P:\lofi-music"  # Update this path to the correct folder
    print(f"Looking for MIDI files in: {midi_folder}")
    midi_files = get_midi_files(midi_folder)
    
    notes = extract_notes(midi_files)
    print(f"Total notes and chords extracted: {len(notes)}")

    sequence_length = 50
    X, y, note_to_int, int_to_note = prepare_sequences(notes, sequence_length)
    print(f"Input shape: {X.shape}, Output shape: {y.shape}")

    model = build_model((X.shape[1], X.shape[2]), y.shape[1])
    model.fit(X, y, epochs=200, batch_size=32)

    generated_notes = generate_music(model, sequence_length, note_to_int, int_to_note)
    print("Generated Notes:", generated_notes)

    create_midi(generated_notes, file_name="generated_lofi_music.mid")
    print("MIDI file generated: generated_lofi_music.mid")
except Exception as e:
    print(f"An error occurred: {e}")
